In [71]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/databased/database/solvency_ii/solvency_ii.sqlite
/kaggle/input/databased/database/solvency_ii/schema.sql
/kaggle/input/databased/database/voter_1/voter_1.sqlite
/kaggle/input/databased/database/concert_singer/concert_singer.sqlite
/kaggle/input/databased/database/concert_singer/schema.sql
/kaggle/input/databased/database/apartment_rentals/apartment_rentals.sqlite
/kaggle/input/databased/database/apartment_rentals/schema.sql
/kaggle/input/databased/database/flight_1/flight_1.sqlite
/kaggle/input/databased/database/flight_1/schema.sql
/kaggle/input/databased/database/gymnast/gymnast.sqlite
/kaggle/input/databased/database/gymnast/schema.sql
/kaggle/input/databased/database/epinions_1/epinions_1.sqlite
/kaggle/input/databased/database/local_govt_and_lot/local_govt_and_lot.sqlite
/kaggle/input/databased/database/local_govt_and_lot/schema.sql
/kaggle/input/databased/database/assets_maintenance/assets_maintenance.sqlite
/kaggle/input/databased/database/assets_maintenance/schem

In [72]:
!pip install rapidfuzz

import difflib
from typing import List, Optional, Tuple
from rapidfuzz import fuzz
import sqlite3
import functools

# fmt: off
_stopwords = {'who', 'ourselves', 'down', 'only', 'were', 'him', 'at', "weren't", 'has', 'few', "it's", 'm', 'again',
              'd', 'haven', 'been', 'other', 'we', 'an', 'own', 'doing', 'ma', 'hers', 'all', "haven't", 'in', 'but',
              "shouldn't", 'does', 'out', 'aren', 'you', "you'd", 'himself', "isn't", 'most', 'y', 'below', 'is',
              "wasn't", 'hasn', 'them', 'wouldn', 'against', 'this', 'about', 'there', 'don', "that'll", 'a', 'being',
              'with', 'your', 'theirs', 'its', 'any', 'why', 'now', 'during', 'weren', 'if', 'should', 'those', 'be',
              'they', 'o', 't', 'of', 'or', 'me', 'i', 'some', 'her', 'do', 'will', 'yours', 'for', 'mightn', 'nor',
              'needn', 'the', 'until', "couldn't", 'he', 'which', 'yourself', 'to', "needn't", "you're", 'because',
              'their', 'where', 'it', "didn't", 've', 'whom', "should've", 'can', "shan't", 'on', 'had', 'have',
              'myself', 'am', "don't", 'under', 'was', "won't", 'these', 'so', 'as', 'after', 'above', 'each', 'ours',
              'hadn', 'having', 'wasn', 's', 'doesn', "hadn't", 'than', 'by', 'that', 'both', 'herself', 'his',
              "wouldn't", 'into', "doesn't", 'before', 'my', 'won', 'more', 'are', 'through', 'same', 'how', 'what',
              'over', 'll', 'yourselves', 'up', 'mustn', "mustn't", "she's", 're', 'such', 'didn', "you'll", 'shan',
              'when', "you've", 'themselves', "mightn't", 'she', 'from', 'isn', 'ain', 'between', 'once', 'here',
              'shouldn', 'our', 'and', 'not', 'too', 'very', 'further', 'while', 'off', 'couldn', "hasn't", 'itself',
              'then', 'did', 'just', "aren't"}
# fmt: on

_commonwords = {"no", "yes", "many"}


def is_number(s: str) -> bool:
    try:
        float(s.replace(",", ""))
        return True
    except:
        return False


def is_stopword(s: str) -> bool:
    return s.strip() in _stopwords


def is_commonword(s: str) -> bool:
    return s.strip() in _commonwords


def is_common_db_term(s: str) -> bool:
    return s.strip() in ["id"]


class Match(object):
    def __init__(self, start: int, size: int) -> None:
        self.start = start
        self.size = size


def is_span_separator(c: str) -> bool:
    return c in "'\"()`,.?! "


def split(s: str) -> List[str]:
    return [c.lower() for c in s.strip()]


def prefix_match(s1: str, s2: str) -> bool:
    i, j = 0, 0
    for i in range(len(s1)):
        if not is_span_separator(s1[i]):
            break
    for j in range(len(s2)):
        if not is_span_separator(s2[j]):
            break
    if i < len(s1) and j < len(s2):
        return s1[i] == s2[j]
    elif i >= len(s1) and j >= len(s2):
        return True
    else:
        return False


def get_effective_match_source(s: str, start: int, end: int) -> Match:
    _start = -1

    for i in range(start, start - 2, -1):
        if i < 0:
            _start = i + 1
            break
        if is_span_separator(s[i]):
            _start = i
            break

    if _start < 0:
        return None

    _end = -1
    for i in range(end - 1, end + 3):
        if i >= len(s):
            _end = i - 1
            break
        if is_span_separator(s[i]):
            _end = i
            break

    if _end < 0:
        return None

    while _start < len(s) and is_span_separator(s[_start]):
        _start += 1
    while _end >= 0 and is_span_separator(s[_end]):
        _end -= 1

    return Match(_start, _end - _start + 1)


def get_matched_entries(
    s: str, field_values: List[str], m_theta: float = 0.85, s_theta: float = 0.85
) -> Optional[List[Tuple[str, Tuple[str, str, float, float, int]]]]:
    if not field_values:
        return None

    if isinstance(s, str):
        n_grams = split(s)
    else:
        n_grams = s

    matched = dict()
    for field_value in field_values:
        if not isinstance(field_value, str):
            continue
        fv_tokens = split(field_value)
        sm = difflib.SequenceMatcher(None, n_grams, fv_tokens)
        match = sm.find_longest_match(0, len(n_grams), 0, len(fv_tokens))
        if match.size > 0:
            source_match = get_effective_match_source(
                n_grams, match.a, match.a + match.size
            )
            if source_match and source_match.size > 1:
                match_str = field_value[match.b : match.b + match.size]
                source_match_str = s[
                    source_match.start : source_match.start + source_match.size
                ]
                c_match_str = match_str.lower().strip()
                c_source_match_str = source_match_str.lower().strip()
                c_field_value = field_value.lower().strip()
                if (
                    c_match_str
                    and not is_number(c_match_str)
                    and not is_common_db_term(c_match_str)
                ):
                    if (
                        is_stopword(c_match_str)
                        or is_stopword(c_source_match_str)
                        or is_stopword(c_field_value)
                    ):
                        continue
                    if c_source_match_str.endswith(c_match_str + "'s"):
                        match_score = 1.0
                    else:
                        if prefix_match(c_field_value, c_source_match_str):
                            match_score = (
                                fuzz.ratio(c_field_value, c_source_match_str) / 100
                            )
                        else:
                            match_score = 0
                    if (
                        is_commonword(c_match_str)
                        or is_commonword(c_source_match_str)
                        or is_commonword(c_field_value)
                    ) and match_score < 1:
                        continue
                    s_match_score = match_score
                    if match_score >= m_theta and s_match_score >= s_theta:
                        if field_value.isupper() and match_score * s_match_score < 1:
                            continue
                        matched[match_str] = (
                            field_value,
                            source_match_str,
                            match_score,
                            s_match_score,
                            match.size,
                        )
    
    if not matched:
        return None
    else:
        return sorted(
            matched.items(),
            key=lambda x: (1e16 * x[1][2] + 1e8 * x[1][3] + x[1][4]),
            reverse=True,
        )


@functools.lru_cache(maxsize=1000, typed=False)
def get_column_picklist(table_name: str, column_name: str, db_path: str) -> list:
    fetch_sql = "SELECT DISTINCT `{}` FROM `{}`".format(column_name, table_name)
    try:
        conn = sqlite3.connect(db_path)
        conn.text_factory = bytes
        c = conn.cursor()
        c.execute(fetch_sql)
        picklist = set()
        for x in c.fetchall():
            if isinstance(x[0], str):
                picklist.add(x[0].encode("utf-8"))
            elif isinstance(x[0], bytes):
                try:
                    picklist.add(x[0].decode("utf-8"))
                except UnicodeDecodeError:
                    picklist.add(x[0].decode("latin-1"))
            else:
                picklist.add(x[0])
        picklist = list(picklist)
    except Exception as e:
        picklist = []
    finally:
        conn.close()
    return picklist


def get_database_matches(
    question: str,
    table_name: str,
    column_name: str,
    db_path: str,
    top_k_matches: int = 2,
    match_threshold: float = 0.85,
) -> List[str]:
    picklist = get_column_picklist(
        table_name=table_name, column_name=column_name, db_path=db_path
    )
    # only maintain data in ``str'' type
    picklist = [ele.strip() for ele in picklist if isinstance(ele, str)]
    # picklist is unordered, we sort it to ensure the reproduction stability
    picklist = sorted(picklist)
    
    matches = []
    if picklist and isinstance(picklist[0], str):
        matched_entries = get_matched_entries(
            s=question,
            field_values=picklist,
            m_theta=match_threshold,
            s_theta=match_threshold,
        )

        if matched_entries:
            num_values_inserted = 0
            for _match_str, (
                field_value,
                _s_match_str,
                match_score,
                s_match_score,
                _match_size,
            ) in matched_entries:
                if "name" in column_name and match_score * s_match_score < 1:
                    continue
                if table_name != "sqlite_sequence":  # Spider database artifact
                    matches.append(field_value.strip())
                    num_values_inserted += 1
                    if num_values_inserted >= top_k_matches:
                        break
    return matches

In [73]:
from collections import defaultdict

def extract_columns(sql, column_names, table_names, aliases=None, from_table_idxs=None):
    used_tables = defaultdict(set)
    current_from_table_idxs = set()
    alias_to_table_idx = {}

    def resolve_col(col_unit):
        if col_unit is None:
            return
        _, col_idx, _ = col_unit
        if col_idx == 0:  # wildcard
            for t_idx, col_name in column_names:
                if t_idx in from_table_idxs:
                    table_name = table_names[t_idx]
                    used_tables[table_name].add(col_name)
        elif col_idx > 0:
            t_idx, col_name = column_names[col_idx]
            if t_idx >= 0:
                table_name = table_names[t_idx]
                used_tables[table_name].add(col_name)

    def resolve_val_unit(val_unit):
        if val_unit is None:
            return
        _, col_unit1, col_unit2 = val_unit
        resolve_col(col_unit1)
        resolve_col(col_unit2)

    def resolve_val(val):
        if isinstance(val, dict):  # subquery
            sub_from_idxs = get_from_table_idxs(val)
            sub_used = extract_columns(val, column_names, table_names, {}, sub_from_idxs)
            for tbl, cols in sub_used.items():
                used_tables[tbl].update(cols)
        elif isinstance(val, (tuple, list)) and len(val) == 3:
            resolve_col(tuple(val))  # Convert list to tuple

    def resolve_condition(conds):
        if not conds:
            return
        if isinstance(conds[0], list):
            for cond in conds:
                if isinstance(cond, list) and len(cond) == 5:
                    not_op, op_id, val_unit, val1, val2 = cond
                    resolve_val_unit(val_unit)
                    resolve_val(val1)
                    resolve_val(val2)
        else:
            not_op, op_id, val_unit, val1, val2 = conds
            resolve_val_unit(val_unit)
            resolve_val(val1)
            resolve_val(val2)

    def resolve_table_units(table_units):
        for unit in table_units:
            type_, val = unit
            if type_ == "table_unit":
                if isinstance(val, int):
                    current_from_table_idxs.add(val)
                    table_name = table_names[val]
                    alias_to_table_idx[str(val)] = val
                elif isinstance(val, dict):  # subquery
                    sub_from_idxs = get_from_table_idxs(val)
                    sub_used = extract_columns(val, column_names, table_names, {}, sub_from_idxs)
                    for tbl, cols in sub_used.items():
                        used_tables[tbl].update(cols)
            elif type_ == "sql":  # e.g., derived table
                sub_from_idxs = get_from_table_idxs(val)
                sub_used = extract_columns(val, column_names, table_names, {}, sub_from_idxs)
                for tbl, cols in sub_used.items():
                    used_tables[tbl].update(cols)

    def get_from_table_idxs(sql_dict):
        from_ = sql_dict.get("from", {})
        idxs = set()
        for unit in from_.get("table_units", []):
            type_, val = unit
            if type_ == "table_unit" and isinstance(val, int):
                idxs.add(val)
        return idxs

    # FROM clause
    from_ = sql.get("from", {})
    table_units = from_.get("table_units", [])
    resolve_table_units(table_units)
    resolve_condition(from_.get("conds", []))
    
    # ✅ Set this before using resolve_col
    if from_table_idxs is None:
        from_table_idxs = current_from_table_idxs
    
    # SELECT
    select_clause = sql.get("select", [False, []])
    if select_clause and isinstance(select_clause, list):
        _, select_cols = select_clause
        for agg_id, val_unit in select_cols:
            resolve_val_unit(val_unit)

    # WHERE clause
    resolve_condition(sql.get("where", []))

    # GROUP BY clause
    for col_unit in sql.get("groupBy", []):
        resolve_col(col_unit)

    # HAVING clause
    resolve_condition(sql.get("having", []))

    # ORDER BY clause
    order_by = sql.get("orderBy")
    if order_by:
        _, val_units = order_by
        for val_unit in val_units:
            resolve_val_unit(val_unit)

    # INTERSECT / UNION / EXCEPT
    for key in ["intersect", "union", "except"]:
        if sql.get(key):
            sub_from_idxs = get_from_table_idxs(sql[key])
            sub_used = extract_columns(sql[key], column_names, table_names, {}, sub_from_idxs)
            for tbl, cols in sub_used.items():
                used_tables[tbl].update(cols)

    return used_tables


In [74]:
def format_used_columns(used_tables):
    return " | ".join(f"{table}: {' , '.join(sorted(cols))}" for table, cols in used_tables.items())

tables_path = "/kaggle/input/spider-data/tables.json"
train_path = "/kaggle/input/spider-data/train_spider.json"
with open(tables_path, "r") as f:
    tables = {tbl["db_id"]: tbl for tbl in json.load(f)}

with open(train_path, "r") as f:
    train = json.load(f)

for i, item in enumerate(train[40:50]):
    sql = item["sql"]
    db_id = item["db_id"]
    query = item["query"]
    table_entry = tables[db_id]
    column_names = table_entry["column_names_original"]
    table_names = table_entry["table_names_original"]
    aliases = {}

    used = extract_columns(sql, column_names, table_names, aliases)
    formatted = format_used_columns(used).lower()

    print(f"\nExample {i+1} ({query}\n{db_id}):\n{formatted}")


Example 1 (SELECT T2.Theme FROM city AS T1 JOIN farm_competition AS T2 ON T1.City_ID  =  T2.Host_city_ID WHERE T1.Population  >  1000
farm):
city: city_id , population | farm_competition: host_city_id , theme

Example 2 (SELECT T2.Theme FROM city AS T1 JOIN farm_competition AS T2 ON T1.City_ID  =  T2.Host_city_ID WHERE T1.Population  >  1000
farm):
city: city_id , population | farm_competition: host_city_id , theme

Example 3 (SELECT Status ,  avg(Population) FROM city GROUP BY Status
farm):
city: population , status

Example 4 (SELECT Status ,  avg(Population) FROM city GROUP BY Status
farm):
city: population , status

Example 5 (SELECT Status FROM city GROUP BY Status ORDER BY COUNT(*) ASC
farm):
city: area_km_2 , census_ranking , city_id , official_name , population , status

Example 6 (SELECT Status FROM city GROUP BY Status ORDER BY COUNT(*) ASC
farm):
city: area_km_2 , census_ranking , city_id , official_name , population , status

Example 7 (SELECT Status FROM city GROUP BY Sta

In [75]:
def extract_skeleton(sql, db_schema):
    table_names_original, table_dot_column_names_original, column_names_original = [], [], []
    for table in db_schema["schema_items"]:
        table_name_original = table["table_name_original"]
        table_names_original.append(table_name_original)

        for column_name_original in ["*"]+table["column_names_original"]:
            table_dot_column_names_original.append(table_name_original+"."+column_name_original)
            column_names_original.append(column_name_original)
    
    parsed_sql = Parser(sql)
    new_sql_tokens = []
    for token in parsed_sql.tokens:
        # mask table names
        if token.value in table_names_original:
            new_sql_tokens.append("_")
        # mask column names
        elif token.value in column_names_original \
            or token.value in table_dot_column_names_original:
            new_sql_tokens.append("_")
        # mask string values
        elif token.value.startswith("'") and token.value.endswith("'"):
            new_sql_tokens.append("_")
        # mask positive int number
        elif token.value.isdigit():
            new_sql_tokens.append("_")
        # mask negative int number
        elif isNegativeInt(token.value):
            new_sql_tokens.append("_")
        # mask float number
        elif isFloat(token.value):
            new_sql_tokens.append("_")
        else:
            new_sql_tokens.append(token.value.strip())

    sql_skeleton = " ".join(new_sql_tokens)
    
    # remove JOIN ON keywords
    sql_skeleton = sql_skeleton.replace("on _ = _ and _ = _", "on _ = _")
    sql_skeleton = sql_skeleton.replace("on _ = _ or _ = _", "on _ = _")
    sql_skeleton = sql_skeleton.replace(" on _ = _", "")
    pattern3 = re.compile("_ (?:join _ ?)+")
    sql_skeleton = re.sub(pattern3, "_ ", sql_skeleton)

    # "_ , _ , ..., _" -> "_"
    while("_ , _" in sql_skeleton):
        sql_skeleton = sql_skeleton.replace("_ , _", "_")
    
    # remove clauses in WHERE keywords
    ops = ["=", "!=", ">", ">=", "<", "<="]
    for op in ops:
        if "_ {} _".format(op) in sql_skeleton:
            sql_skeleton = sql_skeleton.replace("_ {} _".format(op), "_")
    while("where _ and _" in sql_skeleton or "where _ or _" in sql_skeleton):
        if "where _ and _"in sql_skeleton:
            sql_skeleton = sql_skeleton.replace("where _ and _", "where _")
        if "where _ or _" in sql_skeleton:
            sql_skeleton = sql_skeleton.replace("where _ or _", "where _")

    # remove additional spaces in the skeleton
    while "  " in sql_skeleton:
        sql_skeleton = sql_skeleton.replace("  ", " ")

    return sql_skeleton


def get_db_contents(question, table_name_original, column_names_original, db_id, db_path):
    matched_contents = []
    # extract matched contents for each column
    for column_name_original in column_names_original:
        matches = get_database_matches(
            question, 
            table_name_original, 
            column_name_original, 
            db_path + "/{}/{}.sqlite".format(db_id, db_id)
        )
        matches = sorted(matches)
        matched_contents.append(matches)
    
    return matched_contents

def get_db_schemas(all_db_infos):
    db_schemas = {}

    for db in all_db_infos:
        table_names_original = db["table_names_original"]
        table_names = db["table_names"]
        column_names_original = db["column_names_original"]
        column_names = db["column_names"]
        column_types = db["column_types"]

        db_schemas[db["db_id"]] = {}
        
        primary_keys, foreign_keys = [], []
        # record primary keys
        for pk_column_idx in db["primary_keys"]:
            pk_table_name_original = table_names_original[column_names_original[pk_column_idx][0]]
            pk_column_name_original = column_names_original[pk_column_idx][1]
            
            primary_keys.append(
                {
                    "table_name_original": pk_table_name_original.lower(), 
                    "column_name_original": pk_column_name_original.lower()
                }
            )

        db_schemas[db["db_id"]]["pk"] = primary_keys

        # record foreign keys
        for source_column_idx, target_column_idx in db["foreign_keys"]:
            fk_source_table_name_original = table_names_original[column_names_original[source_column_idx][0]]
            fk_source_column_name_original = column_names_original[source_column_idx][1]

            fk_target_table_name_original = table_names_original[column_names_original[target_column_idx][0]]
            fk_target_column_name_original = column_names_original[target_column_idx][1]
            
            foreign_keys.append(
                {
                    "source_table_name_original": fk_source_table_name_original.lower(),
                    "source_column_name_original": fk_source_column_name_original.lower(),
                    "target_table_name_original": fk_target_table_name_original.lower(),
                    "target_column_name_original": fk_target_column_name_original.lower(),
                }
            )
        db_schemas[db["db_id"]]["fk"] = foreign_keys

        db_schemas[db["db_id"]]["schema_items"] = []
        for idx, table_name_original in enumerate(table_names_original):
            column_names_original_list = []
            column_names_list = []
            column_types_list = []
            
            for column_idx, (table_idx, column_name_original) in enumerate(column_names_original):
                if idx == table_idx:
                    column_names_original_list.append(column_name_original.lower())
                    column_names_list.append(column_names[column_idx][1].lower())
                    column_types_list.append(column_types[column_idx])
            
            db_schemas[db["db_id"]]["schema_items"].append({
                "table_name_original": table_name_original.lower(),
                "table_name": table_names[idx].lower(), 
                "column_names": column_names_list, 
                "column_names_original": column_names_original_list,
                "column_types": column_types_list
            })

    return db_schemas


def normalization(sql):
    def white_space_fix(s):
        parsed_s = Parser(s)
        s = " ".join([token.value for token in parsed_s.tokens])

        return s

    # convert everything except text between single quotation marks to lower case
    def lower(s):
        in_quotation = False
        out_s = ""
        for char in s:
            if in_quotation:
                out_s += char
            else:
                out_s += char.lower()
            
            if char == "'":
                if in_quotation:
                    in_quotation = False
                else:
                    in_quotation = True
        
        return out_s
    
    # remove ";"
    def remove_semicolon(s):
        if s.endswith(";"):
            s = s[:-1]
        return s

    # double quotation -> single quotation 
    def double2single(s):
        return s.replace("\"", "'") 
    
    def add_asc(s):
        pattern = re.compile(r'order by (?:\w+ \( \S+ \)|\w+\.\w+|\w+)(?: (?:\+|\-|\<|\<\=|\>|\>\=) (?:\w+ \( \S+ \)|\w+\.\w+|\w+))*')
        if "order by" in s and "asc" not in s and "desc" not in s:
            for p_str in pattern.findall(s):
                s = s.replace(p_str, p_str + " asc")

        return s

    def remove_table_alias(s):
        tables_aliases = Parser(s).tables_aliases
        new_tables_aliases = {}
        for i in range(1,11):
            if "t{}".format(i) in tables_aliases.keys():
                new_tables_aliases["t{}".format(i)] = tables_aliases["t{}".format(i)]
        
        tables_aliases = new_tables_aliases
        for k, v in tables_aliases.items():
            s = s.replace("as " + k + " ", "")
            s = s.replace(k, v)
        
        return s
    
    processing_func = lambda x : (add_asc(lower(white_space_fix(double2single(remove_semicolon(x))))))

    # processing_func = lambda x : remove_table_alias(add_asc(lower(white_space_fix(double2single(remove_semicolon(x))))))
    
    return processing_func(sql)

# extract the skeleton of sql and natsql
def extract_skeleton(sql, db_schema):
    table_names_original, table_dot_column_names_original, column_names_original = [], [], []
    for table in db_schema["schema_items"]:
        table_name_original = table["table_name_original"]
        table_names_original.append(table_name_original)

        for column_name_original in ["*"]+table["column_names_original"]:
            table_dot_column_names_original.append(table_name_original+"."+column_name_original)
            column_names_original.append(column_name_original)
    
    parsed_sql = Parser(sql)
    new_sql_tokens = []
    for token in parsed_sql.tokens:
        # mask table names
        if token.value in table_names_original:
            new_sql_tokens.append("_")
        # mask column names
        elif token.value in column_names_original \
            or token.value in table_dot_column_names_original:
            new_sql_tokens.append("_")
        # mask string values
        elif token.value.startswith("'") and token.value.endswith("'"):
            new_sql_tokens.append("_")
        # mask positive int number
        elif token.value.isdigit():
            new_sql_tokens.append("_")
        # mask negative int number
        elif isNegativeInt(token.value):
            new_sql_tokens.append("_")
        # mask float number
        elif isFloat(token.value):
            new_sql_tokens.append("_")
        else:
            new_sql_tokens.append(token.value.strip())

    sql_skeleton = " ".join(new_sql_tokens)
    
    # remove JOIN ON keywords
    sql_skeleton = sql_skeleton.replace("on _ = _ and _ = _", "on _ = _")
    sql_skeleton = sql_skeleton.replace("on _ = _ or _ = _", "on _ = _")
    sql_skeleton = sql_skeleton.replace(" on _ = _", "")
    pattern3 = re.compile("_ (?:join _ ?)+")
    sql_skeleton = re.sub(pattern3, "_ ", sql_skeleton)

    # "_ , _ , ..., _" -> "_"
    while("_ , _" in sql_skeleton):
        sql_skeleton = sql_skeleton.replace("_ , _", "_")
    
    # remove clauses in WHERE keywords
    ops = ["=", "!=", ">", ">=", "<", "<="]
    for op in ops:
        if "_ {} _".format(op) in sql_skeleton:
            sql_skeleton = sql_skeleton.replace("_ {} _".format(op), "_")
    while("where _ and _" in sql_skeleton or "where _ or _" in sql_skeleton):
        if "where _ and _"in sql_skeleton:
            sql_skeleton = sql_skeleton.replace("where _ and _", "where _")
        if "where _ or _" in sql_skeleton:
            sql_skeleton = sql_skeleton.replace("where _ or _", "where _")

    # remove additional spaces in the skeleton
    while "  " in sql_skeleton:
        sql_skeleton = sql_skeleton.replace("  ", " ")

    return sql_skeleton

In [76]:
# sql_keywords = ['select', 'from', 'where', 'group', 'order', 'limit', 'intersect', 'union', \
#     'except', 'join', 'on', 'as', 'not', 'between', 'in', 'like', 'is', 'exists', 'max', 'min', \
#         'count', 'sum', 'avg', 'and', 'or', 'desc', 'asc']
# import json
# from tqdm import tqdm

# def build_input_output_sequences(tables_path, spider_path, opt):
#     with open(tables_path, 'r') as f:
#         all_db_infos = json.load(f)

#     with open(spider_path, 'r') as f:
#         dataset = json.load(f)

#     db_schemas = get_db_schemas(all_db_infos)
#     input_output_pairs = []

#     for data in tqdm(dataset):
#         question = data["question"].replace("\u2018", "'").replace("\u2019", "'").replace("\u201c", "'").replace("\u201d", "'").strip()
#         db_id = data["db_id"]
#         sql = data["query"].strip()

#         input_sequence = question + " | "
#         output_sequence = ""

#         for table in db_schemas[db_id]["schema_items"]:
#             db_contents = get_db_contents(
#                 question,
#                 table["table_name_original"],
#                 table["column_names_original"],
#                 db_id,
#                 opt.db_path
#             )

#             # Add to input
#             input_sequence += table["table_name_original"] + " : "
#             input_sequence += " , ".join(table["column_names_original"]) + " | "

#             # Add to output
#             output_sequence += table["table_name_original"] + " : "
#             output_sequence += " , ".join(table["column_names_original"]) + " | "

#         # Add foreign keys to input
#         # for fk in db_schemas[db_id]["fk"]:
#         #     input_sequence += (
#         #         fk["source_table_name_original"] + "." + fk["source_column_name_original"]
#         #         + " = " +
#         #         fk["target_table_name_original"] + "." + fk["target_column_name_original"]
#         #         + " | "
#         #     )

#         input_output_pairs.append((input_sequence.strip(), output_sequence.strip()))

#     return input_output_pairs


In [77]:
import json
import torch
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import get_peft_model, LoraConfig, TaskType
from tqdm import tqdm
# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load Data
with open("/kaggle/input/spider-data/train_spider.json", "r") as f:
    train_data = json.load(f)

with open("/kaggle/input/spider-data/tables.json", "r") as f:
    tables_data = json.load(f)
class Opt:
    db_path = "/kaggle/input/databased/database"

opt = Opt()
import json
from tqdm import tqdm

def build_input_output_sequences(tables_path, spider_path, opt):
    with open(tables_path, 'r') as f:
        all_db_infos = json.load(f)

    with open(spider_path, 'r') as f:
        dataset = json.load(f)

    db_schemas = get_db_schemas(all_db_infos)
    input_output_pairs = []

    for data in tqdm(dataset):
        question = data["question"].replace("\u2018", "'").replace("\u2019", "'").replace("\u201c", "'").replace("\u201d", "'").strip()
        db_id = data["db_id"]
        sql = data["query"].strip()

        input_sequence = question + " | "
        sql = data["sql"]
        db_id = data["db_id"]
        query = data["query"]
        table_entry = next(item for item in all_db_infos if item["db_id"] == db_id)
        column_names = table_entry["column_names_original"]
        table_names = table_entry["table_names_original"]
        aliases = {}
    
        used = extract_columns(sql, column_names, table_names, aliases)
        output_sequence = format_used_columns(used).lower()

        for table in db_schemas[db_id]["schema_items"]:
            db_contents = get_db_contents(
                question,
                table["table_name_original"],
                table["column_names_original"],
                db_id,
                opt.db_path
            )

            # Add to input
            input_sequence += table["table_name_original"] + " : "
            input_sequence += " , ".join(table["column_names_original"]) + " | "

        # Add foreign keys to input
        for fk in db_schemas[db_id]["fk"]:
            input_sequence += (
                fk["source_table_name_original"] + "." + fk["source_column_name_original"]
                + " = " +
                fk["target_table_name_original"] + "." + fk["target_column_name_original"]
                + " | "
            )

        input_output_pairs.append((input_sequence.strip(), output_sequence.strip()))
# CAN ALSO KEEP PRIMARY KEY
    return input_output_pairs


pairs = build_input_output_sequences("/kaggle/input/spider-data/tables.json", "/kaggle/input/spider-data/train_spider.json", opt)

for inp, out in pairs[:3]:
    print("INPUT:", inp)
    print("OUTPUT:", out)
    print("=" * 80)


100%|██████████| 7000/7000 [12:54<00:00,  9.04it/s]  

INPUT: How many heads of the departments are older than 56 ? | department : department_id , name , creation , ranking , budget_in_billions , num_employees | head : head_id , name , born_state , age | management : department_id , head_id , temporary_acting | management.head_id = head.head_id | management.department_id = department.department_id |
OUTPUT: head: age , born_state , head_id , name
INPUT: List the name, born state and age of the heads of departments ordered by age. | department : department_id , name , creation , ranking , budget_in_billions , num_employees | head : head_id , name , born_state , age | management : department_id , head_id , temporary_acting | management.head_id = head.head_id | management.department_id = department.department_id |
OUTPUT: head: age , born_state , name
INPUT: List the creation year, name and budget of each department. | department : department_id , name , creation , ranking , budget_in_billions , num_employees | head : head_id , name , born_st

In [78]:
# input_texts = []
# output_texts = []

# for item in train_data:
#     db_id = item['db_id']
#     question = item['question']
#     sql = item['query']
#     db_schema = db_schemas[db_id]

#     table_flags, columns = extract_tables_columns_flags(sql, db_schema)

#     schema_str = "\n".join(f"- {t}: {', '.join(cols)}" for t, cols in db_schema.items())

#     input_text = f"""


# Database Schema:
# {schema_str}

# Question:
# {question}

# Given this, Output 
# Tables : table_name -> True 
# Columns : table_name.column_name
# """

#     table_str = ", ".join([f"{t} -> {str(flag)}" for t, flag in table_flags.items()])
#     column_str = ", ".join(columns)
#     output_text = f"Tables: {table_str}\nColumns: {column_str}"

#     input_texts.append(input_text)
#     output_texts.append(output_text)

#     # print(f"OUTPUT : {output_text}")

# dataset = Dataset.from_dict({
#     "input_text": input_texts,
#     "output_text": output_texts
# })


input_texts, output_texts = zip(*pairs)

# Create HuggingFace Dataset
dataset = Dataset.from_dict({
    "input_text": list(input_texts),
    "output_text": list(output_texts)
})


tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-base")

def tokenize_function(examples):
    model_inputs = tokenizer(examples["input_text"], max_length=512, padding="max_length", truncation=True)
    labels = tokenizer(examples["output_text"], max_length=256, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["input_text", "output_text"])

train_dataloader = DataLoader(
    tokenized_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=lambda x: {
        key: torch.tensor([d[key] for d in x]) for key in x[0]
    }
)
# Model
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-base")
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=16, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-4)

# Training
model.train()
for epoch in range(5):
    total_loss = 0
    for batch in tqdm(train_dataloader,desc="train:"):
        optimizer.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}
    
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"]
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    
        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} - Average Training Loss: {avg_loss:.4f}")

# Save
model.save_pretrained("./model_output")
tokenizer.save_pretrained("./model_output")

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

train:: 100%|██████████| 1750/1750 [09:48<00:00,  2.97it/s]


Epoch 1 - Average Training Loss: 0.1649


train:: 100%|██████████| 1750/1750 [09:48<00:00,  2.98it/s]


Epoch 2 - Average Training Loss: 0.0557


...

train:: 100%|██████████| 1750/1750 [09:47<00:00,  2.98it/s]


Epoch 4 - Average Training Loss: 0.0350


train:: 100%|██████████| 1750/1750 [09:46<00:00,  2.98it/s]


Epoch 5 - Average Training Loss: 0.0286


('./model_output/tokenizer_config.json',
 './model_output/special_tokens_map.json',
 './model_output/spiece.model',
 './model_output/added_tokens.json')

In [79]:
from collections import Counter
import re
def parse_schema_output(text):
    """Robustly parse 'table : col1, col2 | table2:col1,col2' into dict."""
    schema = defaultdict(set)
    parts = text.strip().split("|")

    for part in parts:
        if ":" not in part:
            continue  # skip invalid parts
        table, cols = map(str.strip, part.split(":", 1))
        col_list = re.split(r"\s*,\s*", cols.strip())
        for col in col_list:
            if col:
                schema[table].add(col)
    return schema
from tqdm import tqdm

def evaluate_predictions(pairs, predictions):
    assert len(pairs) == len(predictions), "Mismatch between predictions and ground truth pairs"

    table_correct = 0
    table_total = 0
    table_recall_total = 0

    column_correct = 0
    column_total = 0
    column_recall_total = 0

    print("\nSample Predictions (First 10):\n" + "-"*50)

    for i, ((_, ground_truth), prediction) in enumerate(tqdm(zip(pairs, predictions), total=len(pairs))):
        gt_schema = parse_schema_output(ground_truth)
        pred_schema = parse_schema_output(prediction)

        if i < 10:
            print(f"\nExample {i+1}")
            print("Ground Truth:")
            print(ground_truth)
            print("Prediction:")
            print(prediction)
            print("-" * 50)

        # Evaluate tables
        gt_tables = set(gt_schema.keys())
        pred_tables = set(pred_schema.keys())

        table_correct += len(gt_tables & pred_tables)
        table_total += len(pred_tables)
        table_recall_total += len(gt_tables)

        # Evaluate columns
        for table in gt_tables:
            gt_cols = gt_schema.get(table, set())
            pred_cols = pred_schema.get(table, set())

            column_correct += len(gt_cols & pred_cols)
            column_total += len(pred_cols)
            column_recall_total += len(gt_cols)

    table_accuracy = table_correct / table_total if table_total else 0.0
    table_recall = table_correct / table_recall_total if table_recall_total else 0.0

    column_accuracy = column_correct / column_total if column_total else 0.0
    column_recall = column_correct / column_recall_total if column_recall_total else 0.0

    print("\nFinal Evaluation Metrics\n" + "="*50)
    print(f"Table Accuracy : {table_accuracy:.4f}")
    print(f"Table Recall   : {table_recall:.4f}")
    print(f"Column Accuracy: {column_accuracy:.4f}")
    print(f"Column Recall  : {column_recall:.4f}")


In [80]:
pairs2 = build_input_output_sequences(
    "/kaggle/input/spider-data/tables.json",
    "/kaggle/input/spider-data/dev.json",
    opt
)

inputs = [p[0] for p in pairs2]
gold_outputs = [p[1] for p in pairs2]
from transformers import T5ForConditionalGeneration, T5Tokenizer

# model = T5ForConditionalGeneration.from_pretrained("path/to/your/model")
# tokenizer = T5Tokenizer.from_pretrained("t5-base")  # or your custom tokenizer

model = model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

from tqdm import tqdm
import torch
import random

batch_size = 8  # Try increasing this if you have enough VRAM
num_beams = 6
num_return_sequences = 4

print_indices = set(random.sample(range(len(inputs)), 2))
predictions = []

print("\nGenerating predictions with beam search (batched)...\n")

for start_idx in tqdm(range(0, len(inputs), batch_size)):
    end_idx = min(start_idx + batch_size, len(inputs))
    batch_inputs = inputs[start_idx:end_idx]

    encoded = tokenizer(batch_inputs, return_tensors="pt", truncation=True, padding=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **encoded,
            max_length=256,
            num_beams=num_beams,
            length_penalty = 0.8,
            num_return_sequences=num_return_sequences,
            early_stopping=True
        )

    decoded = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    # Group decoded outputs (num_return_sequences per input)
    grouped = [decoded[i:i + num_return_sequences] for i in range(0, len(decoded), num_return_sequences)]

    for i, group in enumerate(grouped):
        predictions.append(group[0])  # main prediction

        global_idx = start_idx + i
        if global_idx in print_indices:
            print(f"\nBeam Search Results for Example {global_idx + 1}")
            print("-" * 50)
            print("Input:")
            print(inputs[global_idx].strip())
            print("\nBeam Outputs:")
            for j, out in enumerate(group):
                print(f"[Beam {j+1}]: {out}")
            print("-" * 50)

# Evaluation

print(predictions[:5])



100%|██████████| 1034/1034 [00:58<00:00, 17.55it/s] 



Generating predictions with beam search (batched)...



 25%|██▍       | 32/130 [00:33<01:44,  1.06s/it]


Beam Search Results for Example 250
--------------------------------------------------
Input:
What are flight numbers of flights arriving at Airport "APG"? | airlines : uid , airline , abbreviation , country | airports : city , airportcode , airportname , country , countryabbrev | flights : airline , flightno , sourceairport , destairport | flights.destairport = airports.airportcode | flights.sourceairport = airports.airportcode |

Beam Outputs:
[Beam 1]: airports: airportcode , airportname | flights: airline , flightno
[Beam 2]: flights: airline , flightno , sourceairport | airports: airportcode , airportname
[Beam 3]: airports: airportcode , city | flights: airline , flightno
[Beam 4]: airports: airportcode , city | flights: airline , flightno , sourceairport
--------------------------------------------------


 58%|█████▊    | 75/130 [03:06<02:38,  2.88s/it]


Beam Search Results for Example 600
--------------------------------------------------
Input:
What is the content of TV Channel with serial name "Sky Radio"? | tv_channel : id , series_name , country , language , content , pixel_aspect_ratio_par , hight_definition_tv , pay_per_view_ppv , package_option | tv_series : id , episode , air_date , rating , share , 18_49_rating_share , viewers_m , weekly_rank , channel | cartoon : id , title , directed_by , written_by , original_air_date , production_code , channel | tv_series.channel = tv_channel.id | cartoon.channel = tv_channel.id |

Beam Outputs:
[Beam 1]: tv_channel: content , series_name
[Beam 2]: tv_channel: content , id , series_name
[Beam 3]: tv_channel: content , serial_name
[Beam 4]: tv_channel: content , id , serial_name
--------------------------------------------------


100%|██████████| 130/130 [04:30<00:00,  2.08s/it]

['singer: age , country , song_name , song_release_year , is_male , singer_id', 'singer: age , country , song_name , song_release_year , is_male , name', 'singer: age , country', 'singer: age , country', 'singer: age , country']


In [81]:
evaluate_predictions(pairs2, predictions)



Sample Predictions (First 10):
--------------------------------------------------


100%|██████████| 1034/1034 [00:00<00:00, 69055.78it/s]


Example 1
Ground Truth:
singer: age , country , is_male , name , singer_id , song_name , song_release_year
Prediction:
singer: age , country , song_name , song_release_year , is_male , singer_id
--------------------------------------------------

Example 2
Ground Truth:
singer: age , country , is_male , name , singer_id , song_name , song_release_year
Prediction:
singer: age , country , song_name , song_release_year , is_male , name
--------------------------------------------------

Example 3
Ground Truth:
singer: age , country , name
Prediction:
singer: age , country
--------------------------------------------------

Example 4
Ground Truth:
singer: age , country , name
Prediction:
singer: age , country
--------------------------------------------------

Example 5
Ground Truth:
singer: age , country
Prediction:
singer: age , country
--------------------------------------------------

Example 6
Ground Truth:
singer: age , country
Prediction:
singer: age
------------------------------

In [ ]:
!pip install sql_metadata
import re
from sql_metadata import Parser
def isNegativeInt(string):
    if string.startswith("-") and string[1:].isdigit():
        return True
    else:
        return False

def isFloat(string):
    if string.startswith("-"):
        string = string[1:]
    
    s = string.split(".")
    if len(s)>2:
        return False
    else:
        for s_i in s:
            if not s_i.isdigit():
                return False
        return True
def build_input_output_sequences(tables_path, spider_path, opt):
    with open(tables_path, 'r') as f:
        all_db_infos = json.load(f)

    with open(spider_path, 'r') as f:
        dataset = json.load(f)

    db_schemas = get_db_schemas(all_db_infos)
    input_output_pairs = []

    for data in tqdm(dataset):
        question = data["question"].replace("\u2018", "'").replace("\u2019", "'").replace("\u201c", "'").replace("\u201d", "'").strip()
        db_id = data["db_id"]
        sql = data["query"].strip()

        input_sequence = question + " | "
        sql = data["sql"]
        db_id = data["db_id"]
        query = data["query"]
        table_entry = next(item for item in all_db_infos if item["db_id"] == db_id)
        column_names = table_entry["column_names_original"]
        table_names = table_entry["table_names_original"]
        table_names.append("t1")
        table_names.append("t2")
        table_names.append("T1")
        table_names.append("T2")
        aliases = {}
    
        used = extract_columns(sql, column_names, table_names, aliases)
        schema = format_used_columns(used).lower()

        input_sequence += schema
        input_sequence +=' | '

        # Add foreign keys to input
        # for fk in db_schemas[db_id]["fk"]:
        #     input_sequence += (
        #         fk["source_table_name_original"] + "." + fk["source_column_name_original"]
        #         + " = " +
        #         fk["target_table_name_original"] + "." + fk["target_column_name_original"]
        #         + " | "
        #     )
         
        norm_sql = normalization(query).strip()
        sql_skeleton = extract_skeleton(norm_sql, db_schemas[db_id]).strip()
        output_sequence = f"{sql_skeleton} | {norm_sql}"
  
        

        input_output_pairs.append((input_sequence.strip(), output_sequence.strip()))
# CAN ALSO KEEP PRIMARY KEY
    return input_output_pairs

In [ ]:
pairs_train = build_input_output_sequences("/kaggle/input/spider-data/tables.json", "/kaggle/input/spider-data/train_spider.json", opt)

for inp, out in pairs_train[:3]:
    print("INPUT:", inp)
    print("OUTPUT:", out)
    print("=" * 80)

100%|██████████| 7000/7000 [00:16<00:00, 436.12it/s]

INPUT: How many heads of the departments are older than 56 ? | head: age , born_state , head_id , name |
OUTPUT: select count ( _ ) from _ where _ | select count ( * ) from head where age > 56
INPUT: List the name, born state and age of the heads of departments ordered by age. | head: age , born_state , name |
OUTPUT: select _ from _ order by _ asc | select name , born_state , age from head order by age asc
INPUT: List the creation year, name and budget of each department. | department: budget_in_billions , creation , name |
OUTPUT: select _ from _ | select creation , name , budget_in_billions from department


In [84]:
# input_texts = []
# output_texts = []

# for item in train_data:
#     db_id = item['db_id']
#     question = item['question']
#     sql = item['query']
#     db_schema = db_schemas[db_id]

#     table_flags, columns = extract_tables_columns_flags(sql, db_schema)

#     schema_str = "\n".join(f"- {t}: {', '.join(cols)}" for t, cols in db_schema.items())

#     input_text = f"""


# Database Schema:
# {schema_str}

# Question:
# {question}

# Given this, Output 
# Tables : table_name -> True 
# Columns : table_name.column_name
# """

#     table_str = ", ".join([f"{t} -> {str(flag)}" for t, flag in table_flags.items()])
#     column_str = ", ".join(columns)
#     output_text = f"Tables: {table_str}\nColumns: {column_str}"

#     input_texts.append(input_text)
#     output_texts.append(output_text)

#     # print(f"OUTPUT : {output_text}")

# dataset = Dataset.from_dict({
#     "input_text": input_texts,
#     "output_text": output_texts
# })


input_texts, output_texts = zip(*pairs_train)

# Create HuggingFace Dataset
dataset = Dataset.from_dict({
    "input_text": list(input_texts),
    "output_text": list(output_texts)
})


tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

def tokenize_function(examples):
    model_inputs = tokenizer(examples["input_text"], max_length=512, padding="max_length", truncation=True)
    labels = tokenizer(examples["output_text"], max_length=256, padding="max_length", truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["input_text", "output_text"])

train_dataloader = DataLoader(
    tokenized_dataset,
    batch_size=4,
    shuffle=True,
    collate_fn=lambda x: {
        key: torch.tensor([d[key] for d in x]) for key in x[0]
    }
)
# Model
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=16, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model, peft_config)
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-4)

# Training
model.train()
for epoch in range(3):
    total_loss = 0
    for batch in tqdm(train_dataloader,desc="train:"):
        optimizer.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}
    
        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=batch["labels"]
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    
        total_loss += loss.item()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1} - Average Training Loss: {avg_loss:.4f}")

# Save
model.save_pretrained("./encdec_output")
tokenizer.save_pretrained("./encdec_output")

Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

train:: 100%|██████████| 1750/1750 [10:40<00:00,  2.73it/s]


Epoch 1 - Average Training Loss: 1.1132


train:: 100%|██████████| 1750/1750 [10:39<00:00,  2.73it/s]


Epoch 2 - Average Training Loss: 0.0918


train:: 100%|██████████| 1750/1750 [10:40<00:00,  2.73it/s]


Epoch 3 - Average Training Loss: 0.0708


('./encdec_output/tokenizer_config.json',
 './encdec_output/special_tokens_map.json',
 './encdec_output/spiece.model',
 './encdec_output/added_tokens.json')

In [ ]:
pairs2_encdec = build_input_output_sequences(
    "/kaggle/input/spider-data/tables.json",
    "/kaggle/input/spider-data/dev.json",
    opt
)

inputs = [p[0] for p in pairs2_encdec]
gold_outputs = [p[1] for p in pairs2_encdec]
from transformers import T5ForConditionalGeneration, T5Tokenizer

# model = T5ForConditionalGeneration.from_pretrained("path/to/your/model")
# tokenizer = T5Tokenizer.from_pretrained("t5-base")  # or your custom tokenizer

model = model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

from tqdm import tqdm
import torch
import random

batch_size = 8  # Try increasing this if you have enough VRAM
num_beams = 4
num_return_sequences = 4

print_indices = set(random.sample(range(len(inputs)), 2))
predictions = []

print("\nGenerating predictions with beam search (batched)...\n")

for start_idx in tqdm(range(0, len(inputs), batch_size)):
    end_idx = min(start_idx + batch_size, len(inputs))
    batch_inputs = inputs[start_idx:end_idx]

    encoded = tokenizer(batch_inputs, return_tensors="pt", truncation=True, padding=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **encoded,
            max_length=256,
            num_beams=num_beams,
            length_penalty = 0.8,
            num_return_sequences=num_return_sequences,
            early_stopping=True
        )

    decoded = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    # Group decoded outputs (num_return_sequences per input)
    grouped = [decoded[i:i + num_return_sequences] for i in range(0, len(decoded), num_return_sequences)]

    for i, group in enumerate(grouped):
        predictions.append(group[0])  # main prediction

        global_idx = start_idx + i
        if global_idx in print_indices:
            print(f"\nBeam Search Results for Example {global_idx + 1}")
            print("-" * 50)
            print("Input:")
            print(inputs[global_idx].strip())
            print("\nBeam Outputs:")
            for j, out in enumerate(group):
                print(f"[Beam {j+1}]: {out}")
            print("-" * 50)

# Evaluation



In [ ]:
print(predictions[:5])


In [ ]:
norm_sql_list = [pred.split("|", 1)[1].strip() for pred in predictions]

print(norm_sql_list)

In [ ]:
!git clone https://github.com/taoyds/spider.git


In [ ]:
with open("/kaggle/input/spider-data/dev.json", "r") as f:
    val_data = json.load(f)
db_ids = []
labels=[]

for item in val_data:
    db_id = item['db_id']
    sql = item['query']
    labels.append(sql)
    db_ids.append(db_id)
prediction_entries = []
for pred, db_id in zip(norm_sql_list, db_ids):
    prediction_entries.append({
        "query": pred,
        # "question": gold,  # Not used by evaluator
        "db_id": db_id
    })

with open("full.json", "w") as f:
    json.dump(prediction_entries, f, indent=2)    
for i in range(30,50):
    print("="*50)
    print(f"Input:\n{input_texts[i]}")
    print(f"Target:\n{labels[i]}")
    print(f"Prediction:\n{norm_sql_list[i]}")

In [ ]:
import json

# Load gold data from Spider dev.json
with open("/kaggle/input/spider-data/dev.json") as f:
    gold_data = json.load(f)

# Write gold.sql (each line: <GOLD_SQL> \t <DB_ID>)
with open("gold.sql", "w") as f:
    for example in gold_data:
        f.write(f"{example['query']}\t{example['db_id']}\n")

# Write pred.sql (each line: predicted SQL)
# `preds` must be in the same order as `gold_data`
with open("pred.sql", "w") as f:
    for pred in norm_sql_list:
        f.write(pred.strip() + "\n")


In [ ]:
!python3 /kaggle/working/spider/evaluation.py \
    --gold gold.sql \
    --pred pred.sql \
    --etype all \
    --db /kaggle/input/databased/database \
    --table /kaggle/input/spider-data/tables.json